In [1]:
# Agents - AI That Can Think and Act
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate
from langchain import hub
import os
import requests
from datetime import datetime

# Initialize LLM
llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-3.5-turbo",
    temperature=0.1  # Lower temperature for more consistent reasoning
)

print("🤖 Building AI Agents That Can Think and Act")
print("="*50)

# Step 1: Create individual tools
print("🔧 Step 1: Creating Tools...")

# Tool 1: Web Search (using DuckDuckGo as a free alternative)
try:
    search_tool = DuckDuckGoSearchRun()
    print("✅ DuckDuckGo search tool created")
except ImportError:
    print("⚠️ DuckDuckGo tool not available. Install with: pip install duckduckgo-search")
    search_tool = None

# Tool 2: Calculator
def calculator(expression):
    """Safely evaluate mathematical expressions"""
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/()., ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"The result of {expression} is {result}"
        else:
            return "Invalid mathematical expression"
    except Exception as e:
        return f"Error calculating: {e}"

calculator_tool = Tool(
    name="Calculator",
    description="Useful for mathematical calculations. Input should be a mathematical expression like '2+2' or '10*5'.",
    func=calculator
)

# Tool 3: Current Date and Time
def get_current_datetime():
    """Get current date and time"""
    now = datetime.now()
    return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

datetime_tool = Tool(
    name="DateTime",
    description="Get the current date and time",
    func=get_current_datetime
)

# Tool 4: Text Analysis
def analyze_text(text):
    """Analyze text for word count, character count, and readability"""
    if not text or not isinstance(text, str):
        return "Error: Please provide valid text to analyze"
    
    words = len(text.split())
    characters = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    if sentences > 0:
        avg_words_per_sentence = words / sentences
        readability = "Easy" if avg_words_per_sentence < 15 else "Medium" if avg_words_per_sentence < 25 else "Difficult"
    else:
        avg_words_per_sentence = 0
        readability = "Unknown"
    
    return f"""
    Text Analysis Results:
    - Word count: {words}
    - Character count: {characters}
    - Sentence count: {sentences}
    - Average words per sentence: {avg_words_per_sentence:.1f}
    - Readability level: {readability}
    """

text_analysis_tool = Tool(
    name="TextAnalyzer",
    description="Analyze text for statistics like word count, readability, etc. Input should be the text to analyze.",
    func=analyze_text
)

# Tool 5: Simple Data Storage (simulating a database)
stored_data = {}

def store_information(key_value_pair):
    """Store information for later retrieval. Format: 'key: value'"""
    try:
        if ':' not in key_value_pair:
            return "Invalid format. Use 'key: value' format"
        key, value = key_value_pair.split(':', 1)
        key = key.strip()
        value = value.strip()
        stored_data[key] = value
        return f"Stored '{key}' = '{value}'"
    except Exception as e:
        return f"Error storing data: {e}"

def retrieve_information(key):
    """Retrieve stored information by key"""
    key = key.strip()
    if key in stored_data:
        return f"'{key}' = '{stored_data[key]}'"
    else:
        return f"No information found for key '{key}'. Available keys: {list(stored_data.keys())}"

storage_tool = Tool(
    name="DataStorage",
    description="Store information for later use. Input format: 'key: value'",
    func=store_information
)

retrieval_tool = Tool(
    name="DataRetrieval", 
    description="Retrieve previously stored information. Input should be the key name.",
    func=retrieve_information
)

# Combine all tools (only include search if available)
tools = [
    calculator_tool, 
    datetime_tool,
    text_analysis_tool,
    storage_tool,
    retrieval_tool
]

if search_tool:
    tools.insert(0, search_tool)

print("✅ Created tools:", [tool.name for tool in tools])

# Step 2: Create the agent with updated approach
print("\n🔧 Step 2: Initializing Agent...")

# Get the ReAct prompt template from LangChain Hub
try:
    prompt = hub.pull("hwchase17/react")
    print("✅ Retrieved ReAct prompt from LangChain Hub")
except:
    # Fallback to manual prompt if hub is not available
    prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
""")
    print("✅ Using fallback ReAct prompt")

# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=3,
    early_stopping_method="generate",
    handle_parsing_errors=True
)

print("✅ Agent initialized!")

# Step 3: Test the agent with various tasks
print("\n🎯 Step 3: Testing Agent Capabilities")
print("="*50)

test_scenarios = [
    {
        "task": "What's the current date and calculate how many days until New Year 2025?",
        "description": "Multi-step task requiring date lookup and calculation"
    },
    {
        "task": "Analyze this text for readability: 'Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed for every task.'",
        "description": "Text analysis task"
    },
    {
        "task": "Store this information: 'project_deadline: March 15, 2025', then retrieve it and tell me what you stored.",
        "description": "Data storage and retrieval"
    },
    {
        "task": "Calculate 15 * 23 + 47, then analyze the mathematical expression for word count.",
        "description": "Multiple tool usage"
    }
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n🎯 Scenario {i}: {scenario['description']}")
    print(f"📝 Task: {scenario['task']}")
    print("-" * 60)
    
    try:
        result = agent_executor.invoke({"input": scenario['task']})
        print(f"✅ Agent Result: {result['output']}")
    except Exception as e:
        print(f"❌ Agent Error: {e}")
    
    print("="*60)

# Step 4: Demonstrate agent reasoning with a complex task (Multi step request)
print("\n\n🧠 Step 4: Complex Reasoning Task")
print("="*50)

complex_task = """
I need help planning a small project. Here's what I need you to do:
1. Store the project name as 'AI Tutorial Project'  
2. Get the current date
3. Calculate what the date will be 30 days from now
4. Analyze this project description for readability: 'This comprehensive tutorial covers LangChain fundamentals including chains, agents, memory systems, and real-world applications for building production-ready AI systems.'
5. Give me a summary of everything you've found and stored
"""

print(f"🎯 Complex Task: {complex_task}")
print("-" * 50)

try:
    result = agent_executor.invoke({"input": complex_task})
    print(f"🤖 Complex Task Result:")
    print(result['output'])
except Exception as e:
    print(f"❌ Error: {e}")

# Show what data was stored during interactions
print(f"\n📊 Data stored during agent interactions:")
for key, value in stored_data.items():
    print(f"   {key}: {value}")

print("\n✅ Agents demonstration complete!")

print("🔍 How Agents Work:")
print("""
1. OBSERVE: Agent receives a task from the user
2. THINK: Agent decides which tool(s) might be helpful
3. ACT: Agent uses the selected tool
4. OBSERVE: Agent examines the tool's output
5. THINK: Agent decides if more actions are needed
6. REPEAT: Until the task is complete or max iterations reached
""")

# Show what's in our stored data
print(f"\n📊 Data stored during agent interactions:")
for key, value in stored_data.items():
    print(f"   {key}: {value}")

print("\n✅ Agents demonstration complete!")

🤖 Building AI Agents That Can Think and Act
🔧 Step 1: Creating Tools...
⚠️ DuckDuckGo tool not available. Install with: pip install duckduckgo-search
✅ Created tools: ['Calculator', 'DateTime', 'TextAnalyzer', 'DataStorage', 'DataRetrieval']

🔧 Step 2: Initializing Agent...


/Users/thedatayogi/Documents/LB/GenAI_Lab/llms/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


✅ Retrieved ReAct prompt from LangChain Hub
✅ Agent initialized!

🎯 Step 3: Testing Agent Capabilities

🎯 Scenario 1: Multi-step task requiring date lookup and calculation
📝 Task: What's the current date and calculate how many days until New Year 2025?
------------------------------------------------------------


> Entering new AgentExecutor chain...
I need to first get the current date and then calculate the number of days until New Year 2025.
Action: DateTime
Action Input: ❌ Agent Error: get_current_datetime() takes 0 positional arguments but 1 was given

🎯 Scenario 2: Text analysis task
📝 Task: Analyze this text for readability: 'Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions from data without being explicitly programmed for every task.'
------------------------------------------------------------


> Entering new AgentExecutor chain...
I should use the TextAnalyzer tool to analyze the readability of the given text.
Action